In [1]:
import pandas as pd
import numpy as np
from os import path
from CSVUtils import *
import ta
import matplotlib.pyplot as plt
import seaborn as sn
import calendar
from pprint import pprint

In [22]:
DIR = "./from github/Stock-Trading-Environment/data"
nameList = ["^BVSP", "^TWII", "^IXIC"]
df_list = []
startDate = pd.to_datetime("2015-01-01")
endDate = pd.to_datetime("2019-12-31")

for name in nameList:
    df = csv2df(csv_path=DIR, csv_name=name+".csv", source="yahoo")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.dropna()
    df_list.append(df)

In [23]:
twd_df = pd.read_csv("./input/currency/TWD_1995-2020.csv")
brl_df = pd.read_csv("./input/currency/BRL_1995-2020.csv")

twd_df['Date'] = pd.to_datetime(twd_df['Date'])
brl_df['Date'] = pd.to_datetime(brl_df['Date'])

In [24]:
twd_change = twd_df[['Date','Change']]
brl_change = brl_df[['Date','Change']]

In [25]:
def stem_df(df, intersect_dates, startDate, endDate):
    df.dropna(inplace = True)
    df = df[df['Date'].isin(intersect_dates)].reset_index(drop=True)
    df = df[(df['Date'] >= startDate) & (df['Date'] <= endDate)]
    df = df.reset_index(drop=True)
    return df

In [26]:
intersect_dates = df_list[0]['Date']
for df in df_list[1:]+[twd_change,brl_change]:
    df.dropna(inplace = True)
    intersect_dates = np.intersect1d(intersect_dates, df['Date'])

/Users/lzcai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
for i, df in enumerate(df_list):
    df_list[i] = stem_df(df, intersect_dates, startDate, endDate)
twd_change = stem_df(twd_change, intersect_dates, startDate, endDate)
brl_change = stem_df(brl_change, intersect_dates, startDate, endDate)

In [34]:
df_list[0]['FX Change'] = brl_change['Change']
df_list[1]['FX Change'] = twd_change['Change']
df_list[2]['FX Change'] = 0

In [47]:
for i, df in enumerate(df_list):
    df['Cum FX Change'] = (df['FX Change']+1).cumprod()
    df['Actual Price'] = df['Price']*df['Cum FX Change']

In [52]:
df_list[0].head()

,Date,Open,High,Low,Price,Vol,Change,FX Change,Cum FX Change,Actual Price
0,2015-01-05,48512.0,48512.0,47264.0,47517.0,3866100.0,-0.020510,-0.0046,0.995400,47298.421800
1,2015-01-06,47517.0,48061.0,47338.0,48001.0,4559300.0,0.010186,0.0024,0.997789,47894.867869
2,2015-01-07,48006.0,49882.0,48006.0,49463.0,4408800.0,0.030458,0.0062,1.003975,49659.627868
3,2015-01-08,49463.0,50261.0,49017.0,49943.0,3621900.0,0.009704,0.0075,1.011505,50517.597508
4,2015-01-09,49955.0,49955.0,48501.0,48840.0,2999200.0,-0.022085,0.0125,1.024149,50019.431263
